### Global imports

In [11]:
import os
from datetime import datetime
from math import floor
import copy
import torch
import wandb as wb
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import auc, f1_score, roc_curve
from packages.video_utils import H264Extractor, Video
from packages.constants import GOP_SIZE, FRAME_HEIGHT, FRAME_WIDTH, DATASET_ROOT, N_GOPS_FROM_DIFFERENT_DEVICE, N_GOPS_FROM_SAME_DEVICE, SAME_DEVICE_LABEL
from packages.dataset import VisionGOPDataset, GopPairDataset
from packages.common import create_custom_logger
from packages.network import H4vdmNet

### Initialize stuff

In [12]:
if not os.path.exists(DATASET_ROOT):
    raise Exception(f'Dataset root does not exist: {DATASET_ROOT}')

log = create_custom_logger('h4vdm.ipynb')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
log.info(f'Using device: {device}')

h4vdm.ipynb - INFO - Using device: cuda
h4vdm.ipynb - INFO - Using device: cuda
h4vdm.ipynb - INFO - Using device: cuda


### Load GOP dataset

Remember to delete dataset.json if you want to add new devices/videos

In [13]:
bin_path = os.path.abspath(os.path.join(os.getcwd(), 'h264-extractor', 'bin'))
h264_ext_bin = os.path.join(bin_path, 'h264dec_ext_info')
h264_extractor = H264Extractor(bin_filename=h264_ext_bin, cache_dir=DATASET_ROOT)
Video.set_h264_extractor(h264_extractor)

vision_gop_dataset = VisionGOPDataset(
    root_path=DATASET_ROOT,
    devices=[],
    media_types = ['videos'],
    properties=[],
    extensions=['mp4', 'mov', '3gp'],
    gop_size=GOP_SIZE,
    frame_width=FRAME_WIDTH,
    frame_height=FRAME_HEIGHT,
    gops_per_video=4,
    build_on_init=False,
    force_rebuild=False,
    download_on_init=False,
    ignore_local_dataset=False,
    shuffle=False)

is_loaded = vision_gop_dataset.load()
if not is_loaded:
    log.info('Dataset was not loaded. Building...')
else:
    log.info('Dataset was loaded.')

print(f'Dataset length: {len(vision_gop_dataset)}')



h4vdm.ipynb - INFO - Dataset was loaded.
h4vdm.ipynb - INFO - Dataset was loaded.
h4vdm.ipynb - INFO - Dataset was loaded.


Dataset length: 1914


### Create training and testing datasets

In [14]:
from random import shuffle
devices = list(vision_gop_dataset.get_devices())

print(f'{len(devices)} devices: {devices}')

shuffle(devices)
testing_set_1_devices = devices[:len(devices)//2]
training_set_1_devices = devices[len(devices)//2:]

shuffle(devices)
testing_set_2_devices = devices[:len(devices)//2]
training_set_2_devices = devices[len(devices)//2:]

shuffle(devices)
testing_set_3_devices = devices[:len(devices)//2]
training_set_3_devices = devices[len(devices)//2:]

shuffle(devices)
testing_set_4_devices = devices[:len(devices)//2]
training_set_4_devices = devices[len(devices)//2:]

shuffle(devices)
testing_set_5_devices = devices[:len(devices)//3]
training_set_5_devices = devices[len(devices)//3:]

testing_set_6_devices = devices[len(devices)//3:2*(len(devices)//3)]
training_set_6_devices = devices[:len(devices)//3] + devices[2*(len(devices)//3):]

testing_set_7_devices = devices[2*(len(devices)//3):]
training_set_7_devices = devices[:2*(len(devices)//3)]

training_set_devices = [training_set_1_devices, training_set_2_devices, training_set_3_devices, training_set_4_devices, training_set_5_devices, training_set_6_devices, training_set_7_devices]
testing_set_devices = [testing_set_1_devices, testing_set_2_devices, testing_set_3_devices, testing_set_4_devices, testing_set_5_devices, testing_set_6_devices, testing_set_7_devices]

# training_set_devices = [training_set_1_devices]
# testing_set_devices = [testing_set_1_devices]

assert len(training_set_devices) == len(testing_set_devices), 'There must be the same number of training and testing sets'
n_datasets = len(training_set_devices)

for epoch in range(n_datasets):
    print(f'Training set {epoch+1} devices: {training_set_devices[epoch]}')
    print(f'Testing set {epoch+1} devices: {testing_set_devices[epoch]}')

35 devices: ['D01_Samsung_GalaxyS3Mini', 'D02_Apple_iPhone4s', 'D03_Huawei_P9', 'D04_LG_D290', 'D05_Apple_iPhone5c', 'D06_Apple_iPhone6', 'D07_Lenovo_P70A', 'D08_Samsung_GalaxyTab3', 'D09_Apple_iPhone4', 'D10_Apple_iPhone4s', 'D11_Samsung_GalaxyS3', 'D12_Sony_XperiaZ1Compact', 'D13_Apple_iPad2', 'D14_Apple_iPhone5c', 'D15_Apple_iPhone6', 'D16_Huawei_P9Lite', 'D17_Microsoft_Lumia640LTE', 'D18_Apple_iPhone5c', 'D19_Apple_iPhone6Plus', 'D20_Apple_iPadMini', 'D21_Wiko_Ridge4G', 'D22_Samsung_GalaxyTrendPlus', 'D23_Asus_Zenfone2Laser', 'D24_Xiaomi_RedmiNote3', 'D25_OnePlus_A3000', 'D26_Samsung_GalaxyS3Mini', 'D27_Samsung_GalaxyS5', 'D28_Huawei_P8', 'D29_Apple_iPhone5', 'D30_Huawei_Honor5c', 'D31_Samsung_GalaxyS4Mini', 'D32_OnePlus_A3003', 'D33_Huawei_Ascend', 'D34_Apple_iPhone5', 'D35_Samsung_GalaxyTabA']
Training set 1 devices: ['D25_OnePlus_A3000', 'D15_Apple_iPhone6', 'D19_Apple_iPhone6Plus', 'D10_Apple_iPhone4s', 'D07_Lenovo_P70A', 'D35_Samsung_GalaxyTabA', 'D01_Samsung_GalaxyS3Mini', 'D

Build all GOPs so that cache can be cleaned

In [15]:
# for device in vision_gop_dataset.get_devices():
#     for video_metadata in vision_gop_dataset.dataset[device]:
#         video = vision_gop_dataset._get_video_from_metadata(video_metadata)
#         gops = video.get_gops()

#         Video.h264_extractor.clean_cache()
#         video = None
#         gops = None

### Define network parameters and functions

In [16]:
BATCH_SIZE = 72
LEARNING_RATE = 8e-6
LEARNING_RATE_DECAY_FACTOR = 0.97
VALIDATION_PERCENTAGE = 12.5 # 1/8
TEST_PERCENTAGE = 40
WARM_UP_EPOCHS = 5
START_LINEAR_LEARNING_RATE_COEFFICIENT = 1e-3 # basically zero
END_LINEAR_LEARNING_RATE_COEFFICIENT = 1
TARGET_FPR = 0.01
AUC_INCREASE_THRESHOLD = 0.01
AUC_STABLE_COUNTER_THRESHOLD = 3

# define loss function
compute_loss = torch.nn.BCELoss()
# compute_loss = torch.nn.BCEWithLogitsLoss()

# Check if the loss function is working
print(f'Same / Same: {compute_loss(torch.tensor([1.0]), torch.tensor([1.0]))}')
print(f'Same / Different: {compute_loss(torch.tensor([1.0]), torch.tensor([0.0]))}')
print(f'Different / Different: {compute_loss(torch.tensor([0.0]), torch.tensor([0.0]))}')
print(f'Different / Same: {compute_loss(torch.tensor([0.0]), torch.tensor([1.0]))}')

# instantiate the model
net = H4vdmNet()
# net = torch.load('models/2024-03-10_02:17_h4vdm.pth')
net = net.to(device)
  
# instantiate the optimizer
optimizer = torch.optim.Adam(net.parameters(), lr=LEARNING_RATE)

def compute_similarity(gop1_features, gop2_features):
    diff = torch.subtract(gop1_features, gop2_features)
    norm = torch.norm(diff, 2)
    tanh = torch.tanh(norm)
    return (torch.ones(tanh.shape) - tanh)

def train_one_step(model, gop1, gop2, label):
    gop1_features = model(gop1, debug=False, device=device)
    gop2_features = model(gop2, debug=False, device=device)
    gop1_features = gop1_features.to(device)
    gop2_features = gop2_features.to(device)

    similarity = compute_similarity(gop1_features, gop2_features).double()
    similarity.to(device)
    
    label = torch.tensor(label, dtype=float, requires_grad=False, device=device)
    label = label.double()
    
    loss = compute_loss(similarity, label)
    loss = loss.to(device)
    # print(f'Iteration {i}/{len(dataset)} | \tLabel: {label} - Similarity: {similarity} - Loss: {loss}')
    loss /= BATCH_SIZE
    loss.backward()
    return loss

def train_one_batch(model, training_set, optimizer, scheduler):
    total_loss = 0
    for i in range(len(training_set)):         
        gop1, gop2, label = training_set[i]
        loss = train_one_step(model, gop1, gop2, label)
        total_loss += loss.item()
        wb.log({"instantaneous-loss": loss.item()})
        wb.log({"total-loss": total_loss})
        wb.log({"learning-rate": scheduler.get_last_lr()[0]})

        if i % BATCH_SIZE == 0 or i == len(training_set) - 1:
            # update the weights
            optimizer.step()

            # zero the gradients
            optimizer.zero_grad()
    return total_loss

def train_one_epoch(model, devices, optimizer, scheduler):
    model.train()
    total_loss = 0
    for i, training_set_devices in enumerate(devices):
        print(f'Loading training set {i+1}/{len(devices)}')
        training_set = GopPairDataset(vision_gop_dataset, N_GOPS_FROM_SAME_DEVICE, N_GOPS_FROM_DIFFERENT_DEVICE, consider_devices=training_set_devices, shuffle=True)
        print(f'Training batch {i+1}/{len(devices)}')
        train_one_batch(model, training_set, optimizer, scheduler)

    print(f'Updating learning rate from {scheduler.get_last_lr()[0]}', end='')
    scheduler.step()
    print(f' to {scheduler.get_last_lr()[0]}')
    
    return total_loss

def validate_one_step(model, gop1, gop2, label):
    gop1_features = model(gop1, debug=False, device=device)
    gop2_features = model(gop2, debug=False, device=device)
    gop1_features = gop1_features.to(device)
    gop2_features = gop2_features.to(device)

    similarity = compute_similarity(gop1_features, gop2_features).double()
    similarity.to(device)
    
    label = torch.tensor(label, dtype=float, requires_grad=False, device=device)
    label = label.double()
    
    loss = compute_loss(similarity, label)

    return loss, label, similarity

def validate_one_epoch(model, devices):
    model.eval()
    labels = []
    similarities = []
    for i, testing_set_devices in enumerate(devices):
        print(f'Loading validation set {i+1}/{len(devices)}')
        testing_set = GopPairDataset(vision_gop_dataset, N_GOPS_FROM_SAME_DEVICE, N_GOPS_FROM_DIFFERENT_DEVICE, consider_devices=testing_set_devices, shuffle=True)
        testing_set.pair_dataset = testing_set.pair_dataset[:floor(len(testing_set)*(1-TEST_PERCENTAGE/100))] # reduce size by removing TEST_PERCENTAGE % of the dataset
        validation_set = copy.deepcopy(testing_set)
        validation_set.pair_dataset = validation_set.pair_dataset[:floor(len(testing_set)*(1-VALIDATION_PERCENTAGE/100))] # VALIDATION_PERCENTAGE % of the training set is used for validation, works because the dataset is shuffled
        testing_set.pair_dataset = testing_set.pair_dataset[floor(len(testing_set)*(1-VALIDATION_PERCENTAGE/100)):]
        
        print(f'Validating batch {i+1}/{len(devices)}')
        for j in range(len(testing_set)):
            gop1, gop2, label = testing_set[j]
            loss, label, similarity = validate_one_step(model, gop1, gop2, label)
            wb.log({"validation-loss": loss.item()})
            labels.append(label.item())
            similarities.append(similarity.item())

    return labels, similarities

def compute_ROC(scores, labels, show: bool = True):
    # compute ROC
    fpr, tpr, thresholds = roc_curve(np.asarray(labels), np.asarray(scores), drop_intermediate=False)
    # compute AUC
    roc_auc = auc(fpr, tpr)

    tnr = 1 - fpr
    max_index = np.argmax(tpr + tnr)

    threshold = thresholds[max_index]
    chosen_tpr = tpr[max_index]
    chosen_fpr = fpr[max_index]


    if show is True:
        lw = 2
        plt.figure()
        plt.title('Receiver Operating Characteristic (ROC)')
        plt.plot(fpr, tpr, lw=1, alpha=0.3, label='ROC (AUC = %0.2f)' % (roc_auc))
        plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Luck')
        plt.plot(chosen_fpr, chosen_tpr, 'o', markersize=10, alpha=0.5, label="Threshold = %0.2f" % threshold)
        plt.xlim([-0.05, 1.05])
        plt.ylim([-0.05, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.legend(loc="lower right")
        plt.show()
    return threshold, chosen_tpr, chosen_fpr, roc_auc

from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

def compute_metrics(scores, labels, threshold):
    thresholded_scores = scores > threshold

    precision = precision_score(labels, thresholded_scores, average='macro')
    recall = recall_score(labels, thresholded_scores, average='macro')
    f1 = f1_score(labels, thresholded_scores, average='macro')
    accuracy = accuracy_score(labels, thresholded_scores)

    return precision, recall, f1, accuracy

Same / Same: 0.0
Same / Different: 100.0
Different / Different: 0.0
Different / Same: 100.0


### Training and testing loop

In [ ]:
# N_GOPS_FROM_DIFFERENT_DEVICE = 3
# N_GOPS_FROM_SAME_DEVICE = 3

wb.init(project='h4vdm', config={"learning-rate": LEARNING_RATE,
                                 "learning-rate-decay-factor": LEARNING_RATE_DECAY_FACTOR,
                                 "start-linear-learning-rate-coefficient": START_LINEAR_LEARNING_RATE_COEFFICIENT,
                                 "end-linear-learning-rate-coefficient": END_LINEAR_LEARNING_RATE_COEFFICIENT,
                                 "n-warmup-epochs": WARM_UP_EPOCHS,
                                 "n_epochs": n_datasets,
                                 "n-gops-from-same-device": N_GOPS_FROM_SAME_DEVICE,
                                 "n-gops-from-different-device": N_GOPS_FROM_DIFFERENT_DEVICE,
                                 "validation-percentage": VALIDATION_PERCENTAGE,
                                 "test-percentage": TEST_PERCENTAGE,
                                 "batch-size": BATCH_SIZE,
                                 "target-fpr": TARGET_FPR,
                                 "auc-increase-threshold": AUC_INCREASE_THRESHOLD})

scheduler = torch.optim.lr_scheduler.LinearLR(optimizer, START_LINEAR_LEARNING_RATE_COEFFICIENT, END_LINEAR_LEARNING_RATE_COEFFICIENT, WARM_UP_EPOCHS)

print('Starting warmup')
for epoch in range(WARM_UP_EPOCHS - 1):
    print(f'Warmup epoch {epoch+1}/{WARM_UP_EPOCHS}')
    
    epoch_loss = train_one_epoch(net, training_set_devices, optimizer, scheduler)
    
    labels, similarities = validate_one_epoch(net, testing_set_devices)

    threshold, tpr, fpr, roc_auc = compute_ROC(similarities, labels)
    precision, recall, f1, accuracy = compute_metrics(similarities, labels, threshold)

    wb.log({"roc-threshold": threshold})
    wb.log({"roc-tpr": tpr})
    wb.log({"roc-fpr": fpr})
    wb.log({"roc-auc": roc_auc})
    wb.log({"f1-score": f1})
    wb.log({"accuracy": accuracy})
    wb.log({"precision": precision})
    wb.log({"recall": recall})

    print(f'Warmup epoch {epoch+1}/{WARM_UP_EPOCHS} done')
    print('')
print('Warmup done\n')

# scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, LEARNING_RATE_DECAY_FACTOR)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, LEARNING_RATE_DECAY_FACTOR, last_epoch=19)

old_roc_auc = 0
auc_stable_counter = 0
while True:
    old_roc_auc = roc_auc
    print(f'Epoch {epoch+1}')
    
    epoch_loss = train_one_epoch(net, training_set_devices, optimizer, scheduler)
    
    labels, similarities = validate_one_epoch(net, testing_set_devices)

    threshold, tpr, fpr, roc_auc = compute_ROC(similarities, labels)
    precision, recall, f1, accuracy = compute_metrics(similarities, labels, threshold)

    wb.log({"roc-threshold": threshold})
    wb.log({"roc-tpr": tpr})
    wb.log({"roc-fpr": fpr})
    wb.log({"roc-auc": roc_auc})
    wb.log({"f1-score": f1})
    wb.log({"accuracy": accuracy})
    wb.log({"precision": precision})
    wb.log({"recall": recall})

    print(f'Epoch {epoch+1} done')
    print('')

    if abs(roc_auc - old_roc_auc) > AUC_INCREASE_THRESHOLD:
        auc_stable_counter += 1
    if auc_stable_counter > AUC_STABLE_COUNTER_THRESHOLD:
        print(f'ROC AUC has not increased for {auc_stable_counter} epochs. Training terminated.')
        print(f'|New ROC AUC: {roc_auc} - Old ROC AUC: {old_roc_auc}| = |{roc_auc - old_roc_auc}| > {AUC_INCREASE_THRESHOLD}')
        break

    if not os.path.exists('config.txt'):
        file = open("config.txt", "w")
        file.close()
    with open('config.txt', 'r') as file:
        first_line = file.readline()
        if 'stop' in first_line:
            print("Stopping...")
            break

    epoch += 1

print(f'Training terminated after {epoch} epochs')

In [ ]:
filename = os.path.join('models', datetime.today().strftime('%Y-%m-%d_%H:%M') + '_h4vdm.pth')
print(f'Saving model to {filename}')
torch.save(net, filename)